In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
KPL = KessanPl()
KPL.with_columns_lastyear_settlement()
df1 = KPL.df.filter(pl.col("settlement_type")=="四")
df1 = df1.with_columns([
    pl.col("settlement_date").alias("key")
])

In [4]:
# para
valuation_date = date(2023, 9, 15)

KPL = KessanPl()
df2 = KPL.get_settlement_forcast_by_diff_growth_rate(valuation_date)
df2 = df2.filter(pl.col("settlement_type")=="四")
df2 = df2.with_columns([
    pl.col("nxt_settlement_date").alias("key")
])

In [5]:
KPL = KessanPl()
df3 = KPL.get_current_settlements(valuation_date)
df3 = df3.filter(pl.col("settlement_type")=="四")
df3 = df3.with_columns([
    pl.col("settlement_date").alias("key")
])

In [6]:
KPL = KessanPl()
df = KPL.get_settlement_updown_rate("四")
df = df.with_columns([
    pl.col("settlement_date").alias("key")
])
df4 = df.select(["code", "key", "updown_rate", "nh_updown_rate"])
# df4.filter(pl.col("code")==9022)

In [7]:
# df1, df2, df3, df4をjoin&select
df = df2.join(df1, on=["code", "key"], how="left")
df21 = df

df = df21.with_columns([
    pl.col("settlement_date").alias("lst_settlement_date"),
    pl.col("announcement_date").alias("lst_announcement_date"),
    pl.col("settlement_type").alias("lst_settlement_type"),
    pl.col("sales").alias("lst_sales"),
    pl.col("operating_income").alias("lst_operating_income"),
    pl.col("ordinary_profit").alias("lst_ordinary_profit"),
    pl.col("final_profit").alias("lst_final_profit"),
    pl.col("quater").alias("lst_quater")
])
df = df.select([
    "code",	
    "lst_settlement_date",
    "lst_announcement_date",
    "lst_settlement_type",
    'lst_sales',
    'lst_operating_income',
    'lst_ordinary_profit',
    'lst_final_profit',
    "quater",
    "ly_settlement_date",
    "ly_sales",
    "ly_operating_income",
    "ly_ordinary_profit",
    "ly_final_profit",
    "nxt_settlement_date",
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",	
    "fcst_dgr_final_profit",
    "key"
])
df = df.join(df3, on=["code", "key"], how="left")
df = df.drop_nulls()

df123 = df

In [8]:
# ly_fcstの売上高、営業利益成長率を
# 列名ly_fcst_sales_gr, ly_fcst_operating_income_grで追加
df = df123.with_columns([
    (pl.lit(100) * pl.col("lst_operating_income") / pl.col("lst_sales")).round(2).alias("lst_operating_income_pr"),
    (pl.lit(100) * (pl.col("fcst_dgr_sales") - pl.col("ly_sales")) / pl.col("ly_sales")).round(2).alias("ly_fcst_sales_gr"),
    (pl.lit(100) * (pl.col("fcst_dgr_operating_income") - pl.col("ly_operating_income")) / pl.col("ly_operating_income")).round(2).alias("ly_fcst_operating_income_gr"),
])

df = df.join(df4, on=["code", "key"], how="left")
df = df.drop_nulls()

scols = [
    "code",
    "lst_settlement_date",
    #lst_announcement_date,
    #lst_settlement_type,
    "lst_sales",
    "lst_operating_income",
    "lst_ordinary_profit",
    "lst_final_profit",
    "quater",
    #ly_settlement_date,
    "ly_sales",
    "ly_operating_income",
    #ly_ordinary_profit,
    #ly_final_profit,
    #nxt_settlement_date,
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",
    "fcst_dgr_final_profit",
    #key,
    #settlement_date,
    #settlement_type,
    #announcement_date,
    "sales",
    "operating_income",
    "ordinary_profit",
    "final_profit",
    #reviced_eps,
    #dividend,
    #quater_right,
    "lst_operating_income_pr",
    "ly_fcst_sales_gr",
    "ly_fcst_operating_income_gr",
    "updown_rate",
    "nh_updown_rate"
]
df = df.select(scols)
codes = df["code"].unique()
df = df.sort(by=["ly_fcst_operating_income_gr"], descending=[True])
original_df = df
original_df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
3777,2023-06-30,4829,197,197,175,2,310,4,100950,5868,5868,5462,4855,99,93,103,4.08,32464.52,146600.0,8.9,2.29
2497,2023-06-30,7224,5361,5366,3685,1,1861,11,10930,8782,8794,5668,1946,7,13,-605,74.21,487.32,79736.36,-4.4,0.45
7987,2023-06-30,13954,-286,-46,-57,1,14702,-1,14387,-415,-91,32,14380,-85,53,275,-2.05,-2.14,41400.0,5.5,2.81
6356,2023-06-30,2155,445,459,298,1,1534,4,2814,758,777,444,2009,358,363,247,20.65,83.44,18850.0,11.5,-7.8
7212,2023-06-30,71199,73,308,-327,1,61449,10,84727,1602,1106,-895,68104,-1363,-1223,-189,0.1,37.88,15920.0,-20.7,1.68
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7678,2023-06-30,1690,59,60,36,1,1518,-1,2016,166,162,117,1822,62,69,45,3.49,32.81,-16700.0,10.3,1.11
5210,2023-06-30,18814,2005,4331,8623,1,17065,-22,20583,3829,9502,17143,18026,1200,1105,1133,10.66,20.62,-17504.55,-12.9,1.02
6249,2023-06-30,9939,3412,3507,2148,1,3201,-67,34637,14897,15189,9335,8995,2663,2721,2078,34.33,982.07,-22334.33,-43.5,2.51


In [9]:
######
###### filter
######

In [10]:
# 昨年同期営業利益マイナス
# 前四半期営業利益マイナス
# 今四半期予想前年同期売上高10%成長
# 今四半期予想営業利益プラス
# 今四半期売上高差分営業利益プラス20%
min_ly_operating_income = -999999999
max_ly_operating_income = 0

min_lst_operating_income_pr = -99999999
max_lst_operating_income_pr = 0

min_ly_fcst_sales_gr = 10
max_ly_fcst_sales_gr = 999999999

min_ly_fcst_operating_income_gr = -999999999	
max_ly_fcst_operating_income_gr	= 0

min_fcst_dgr_operating_income = 20
max_fcst_dgr_operating_income = 999999999


In [11]:
df = original_df

df = df\
    .filter(pl.col("ly_operating_income") >= min_ly_operating_income)\
    .filter(pl.col("ly_operating_income")  < max_ly_operating_income)\
\
    .filter(pl.col("lst_operating_income_pr") >= min_lst_operating_income_pr)\
    .filter(pl.col("lst_operating_income_pr") < max_lst_operating_income_pr)\
\
    .filter(pl.col("ly_fcst_sales_gr") >= min_ly_fcst_sales_gr)\
    .filter(pl.col("ly_fcst_sales_gr") < max_ly_fcst_sales_gr)\
\
    .filter(pl.col("ly_fcst_operating_income_gr") >= min_ly_fcst_operating_income_gr)\
    .filter(pl.col("ly_fcst_operating_income_gr") < max_ly_fcst_operating_income_gr)\
\
    .filter(pl.col("fcst_dgr_operating_income") >= min_fcst_dgr_operating_income)\
    .filter(pl.col("fcst_dgr_operating_income") <  max_fcst_dgr_operating_income)

filtered_df = df
pl.Config(tbl_rows=df.shape[0])
pl.Config.set_tbl_cols(df.shape[1])
df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
3077,2023-06-30,1041,-58,-59,-61,1,854,-263,1170,21,-124,-160,1093,-36,-34,-43,-5.57,37.0,-107.98,-16.4,1.68
3909,2023-06-30,1870,-56,-58,2,2,1055,-189,1990,28,49,139,1066,-98,-99,-55,-2.99,88.63,-114.81,-15.1,2.29
7494,2023-06-30,16144,-2,-165,-201,3,12730,-2237,16225,450,-82,-178,13061,-1669,-1524,-1012,-0.01,27.45,-120.12,-4.8,2.29
4192,2023-06-30,763,-99,-101,-102,2,636,-316,969,65,71,72,827,-74,-80,-83,-12.98,52.36,-120.57,-22.8,1.38
8160,2023-06-30,11647,-232,-217,-360,1,9967,-1036,13464,282,314,0,12218,-164,-163,-52,-1.99,35.09,-127.22,-0.2,1.11
9012,2023-06-30,1209,-29,-14,475,1,1140,-109,1268,35,34,1033,1208,61,57,-241,-2.4,11.23,-132.11,-6.8,0.11
6662,2023-06-30,392,-19,-21,-97,4,170,-75,391,26,22,-129,169,-104,-103,-105,-4.85,130.0,-134.67,-4.8,1.68
7078,2023-06-30,1297,-23,-28,-41,1,1215,-162,2645,58,26,28,1255,-138,-141,-84,-1.77,117.7,-135.8,2.4,1.2
3205,2023-06-30,7080,-114,15,-68,1,6838,-485,8277,191,338,144,6901,-256,-239,-322,-1.61,21.04,-139.38,19.1,2.51


In [12]:
df = original_df

print("##### 全体")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'全銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')
print()

df = filtered_df
print("##### 抽出銘柄")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'抽出銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')

##### 全体
総銘柄数：3627
上昇銘柄数:1580／下落銘柄数:2047／上昇率:(43.6%)
全銘柄平均騰落率:-0.69%
日経225平均騰落率:-0.44%

##### 抽出銘柄
総銘柄数：46
上昇銘柄数:15／下落銘柄数:31／上昇率:(32.6%)
抽出銘柄平均騰落率:-2.71%
日経225平均騰落率:1.27%


In [13]:
#####
##### 個別銘柄の分析
#####

In [38]:
# 四季報
code = 3237
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

3237(イントランス)の四季報データ履歴

発行日: 2023年09月15日
【赤字幅縮小】
  京都市内の民泊施設が首尾よく売れる。赤字続くハーブ事業も売却。ホテル運営受託は人流再開、訪日客需要で息を継ぐ。受託ホテルの開業準備費に開業遅れなど重いが、前号から営業赤字幅は縮小。
【膠着】
  ホテル関連ファンドは出資者決まるも、物件の事前調査で頓挫多く、組成難航中。運営中の一部のホテルをバンヤンツリーブランドに変更、海外富裕層に訴求。

発行日: 2023年06月23日
【水面下】
  訪日客再来でホテル運営受託が息吹き返す。新規獲得も寄与。主力の不動産は賃貸管理安定だが、保有全物件の売却企図し賃収減。受託ホテルの開業費用が負担。固定費重く、営業赤字幅縮小でも水面下。
【手当て】
  第三者割当で転換社債と新株予約権発行し最大で約６億円調達。新規ホテル運営受託やホテル関連ファンド組成に充当。赤字続くハーブ事業の売却もくろむ。

発行日: 2023年03月17日
【底ばい】
  マイタケはライバル増産響き数量・単価弱含む。マッシュルームも振るわず。電力費想定超。減益幅拡大。24年３月期は通年化の高級白マイタケ軸にブナシメジなど数量底上げ。ただ売価戻り鈍く、原燃料高こなせず。販促費増。営業益底ばい。
【模索】
  課題の欧米、ＡＳＥＡＮ進出は、キノコ取り扱うアジア圏以外の企業軸に事業連携模索。店頭メニュー試食提案再開。

発行日: 2022年12月16日
【赤字転落】
  前期貢献した信託受益権の売却益が剥落。主力のホテルは運営受託の獲得に注力するが、コロナ禍での観光需要回復鈍く想定以下。開業の先行費用もあり、前号比で利益大幅減額、一転赤字。24年３月期はホテル運営受託の獲得案件が増加。
【ファンド】
  手元の現預金では大きな新規投資は難しく、ファンド組成に全力。収益源の確保へホテル以外の新規事業も模索。


In [39]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [37]:
# 日足
delta = relativedelta(days=100)

KPL = KessanPl()
df = KPL.df.filter(pl.col("announcement_date")<valuation_date).filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.add_vline(valuation_date)
PFIG.fig.show()


決算発表日: 2023-08-09
